# Supervised Fine-Tuning with LLama-Factory (LoRA/QLoRA)

This notebook demonstrates how to perform supervised fine-tuning using LLama-Factory with LoRA or QLoRA techniques on a small model like Gemma 2B quantized.

## Setup

First, let's clone the LLama-Factory repository and install the necessary dependencies.

In [1]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
!pip install -r requirements.txt

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 17708, done.
remote: Counting objects: 100% (8100/8100), done.
remote: Compressing objects: 100% (695/695), done.
remote: Total 17708 (delta 7612), reused 7504 (delta 7405), pack-reused 9608 (from 1)
Receiving objects: 100% (17708/17708), 226.94 MiB | 15.06 MiB/s, done.
Resolving deltas: 100% (13042/13042), done.
/content/LLaMA-Factory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━

## Hugging Face Authentication

Set up your Hugging Face token to access the model and datasets.

In [3]:
from huggingface_hub import login
login()

## Prepare the Dataset

For this example, we'll use a small subset of the Alpaca dataset for fine-tuning.

In [5]:
!python scripts/data_preprocess.py \
    --dataset alpaca \
    --subset_num 1000 \
    --output_path ./data/alpaca_subset.json

## Fine-tuning

Now, let's perform supervised fine-tuning using LoRA on the Gemma 2B quantized model.

In [6]:
!python src/train_bash.py \
    --stage sft \
    --model_name_or_path google/gemma-2b \
    --do_train \
    --dataset alpaca \
    --dataset_dir ./data \
    --template default \
    --finetuning_type lora \
    --lora_target q_proj,v_proj \
    --output_dir ./output \
    --overwrite_cache \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 1000 \
    --learning_rate 5e-5 \
    --num_train_epochs 3.0 \
    --plot_loss \
    --fp16

## Inference

After fine-tuning, let's test the model with some example prompts.

In [7]:
!python src/cli_demo.py \
    --model_name_or_path google/gemma-2b \
    --template default \
    --finetuning_type lora \
    --checkpoint_dir ./output